# __Second-Order HMM for typos correction__
_Author_ : Xudong ZHANG and Qixiang PENG  
_Date_   : Mon Jan 9 23:53:48 CEST 2017  

In [30]:
from numpy import array, ones, zeros, multiply
import numpy as np
import sys
import pickle
UNK = "<unk>"  # token to map all out-of-vocabulary letters (OOVs)
UNKid = 0      # index for UNK
epsilon = 1e-100
class HMM_2:
    def __init__(self, state_list, observation_list,
                 transition_proba=None,
                 observation_proba=None,
                 initial_state_proba=None, smoothing=0.01):
        """
        Builds a Hidden 2 order Markov Model
        * state_list is the list of state symbols [q_0...q_(N-1)]
        * observation_list is the list of observation symbols [v_0...v_(M-1)]
        * transition_proba is the transition probability matrix
            [a_ijk] a_i(j*N+k) = Pr(Y_(t+1)=q_i|Y_(t-1)=q_j and Y_(t)=q_k
        * observation_proba is the observation probablility matrix
            [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
        * initial_state_proba is the initial state distribution
            [p_ij] p_i*N+j = Pr(Y_0=q_i and Y_1=q_j)
        * attention: We use inverse row and col rather than example in book
        because `/` represent divide by rows in numpy."""
        print("HMM_2 creating with: ")
        self.N = len(state_list)       # number of states
        self.M = len(observation_list)  # number of possible emissions
        print(str(self.N) + " states")
        print(str(self.M) + " observations")
        self.omega_Y = [s for s in state_list]
        self.omega_X = [o for o in observation_list]
        if transition_proba is None:
            self.transition_proba = zeros((self.N, self.N**2), dtype = np.float)
        else:
            self.transition_proba = transition_proba
        if observation_proba is None:
            self.observation_proba = zeros((self.M, self.N**2), dtype = np.float)
        else:
            self.observation_proba = observation_proba
        if initial_state_proba is None:
            self.initial_state_proba = zeros(self.N**2, dtype = np.float)
        else:
            self.initial_state_proba = initial_state_proba
        self.make_indexes()  # build indexes, i.e the mapping between token and int
        self.smoothing = smoothing
    def make_indexes(self):
        """Creates the reverse table that maps states/observations names
        to their index in the probabilities array"""
        self.Y_index = {}
        for i in range(self.N):
            self.Y_index[self.omega_Y[i]] = i
        self.X_index = {}
        for i in range(self.M):
            self.X_index[self.omega_X[i]] = i
        print("Y_index:\n", self.Y_index)
        print("X_index:\n", self.X_index)
        
    def get_observationIndices(self, observations):
        """return observation indices, i.e
        return [self.O_index[o] for o in observations]
        and deals with OOVs
        """
        indices = zeros(len(observations), int)
        k = 0
        for o in observations:
            if o in self.X_index:
                indices[k] = self.X_index[o]
            else:
                indices[k] = UNKid
            k += 1
        return indices
    def data2indices(self, word):
        """From one tagged word of the given corpus:
        - extract the letters and tags
        - returns two list of indices, one for each
        -> (letterids, tagids)
        """
        letterids = list()
        tagids = list()
        for couple in word:
            letter = couple[0]
            tag = couple[1]
            if letter in self.X_index:
                letterids.append(self.X_index[letter])
            else:
                letterids.append(UNKid)
            tagids.append(self.Y_index[tag])
        return letterids, tagids
    def observation_estimation(self, pair_counts):
        """ Build the observation distribution:
            observation_proba is the observation probablility matrix
            b[k,i*N+j] = Pr(X_t=v_k|Y_t=q_j and Y_(t-1)=q_i)"""
        # fill with counts
        for pair in pair_counts:
            letter = pair[1]
            tag = pair[0][1]
            pre_tag = pair[0][0]
            cpt = pair_counts[pair]
            k = 0  # for <unk>
            if letter in self.X_index:
                k = self.X_index[letter]
            i = self.Y_index[pre_tag]
            j = self.Y_index[tag]
            self.observation_proba[k, i * self.N + j] = cpt
        # normalize, smoothing est pour eviter prob(x(t), s(i)) == 0.
        # Apres, on devra faire une normalisation quand meme.
        self.observation_proba = self.observation_proba + self.smoothing
        # smoothing > 0 et apartient Real. il n'est pas obgalitoire de etre
        # inferieur que 1.
        self.observation_proba = self.observation_proba / \
            self.observation_proba.sum(axis=0).reshape(1, self.N**2)
    def transition_estimation(self, trans_counts):
        """ Build the transition distribution:
            transition_proba is the transition matrix with :
            a[k,(i*N+j)] = Pr(Y_(t)=q_k|Y_(t-1)=q_j and Y_(t-2)=q_i)
        """
        # fill with counts
        for pair in trans_counts:
            k = self.Y_index[pair[1]]
            #((q_i, q_j), q_k) <==> ((Y_(t-2), Y_(t-1)), Y_(t))
            i = self.Y_index[pair[0][0]]
            j = self.Y_index[pair[0][1]]
            self.transition_proba[k, (i * self.N + j)] = trans_counts[pair]
        # normalize
        self.transition_proba = self.transition_proba + self.smoothing
        self.transition_proba = self.transition_proba / \
            self.transition_proba.sum(axis=0).reshape(1, self.N**2)
    def init_estimation(self, init_counts):
        """Build the init. distribution
        The initial distribution μ of (Y0,Y1) on S×S, such that,
        for every states (x,y) in S×S, one has P(Y0=x,Y1=y)=μ(x,y)"""
        # fill with counts
        for tag in init_counts:
            i = self.Y_index[tag[0]]
            j = self.Y_index[tag[1]]
            self.initial_state_proba[i * self.N + j] = init_counts[tag]
        # normalize
        self.initial_state_proba = self.initial_state_proba / \
            sum(self.initial_state_proba)
    def supervised_training(self, pair_counts, trans_counts, init_counts):
        """ Train the HMM_2's parameters. This function wraps everything"""
        self.observation_estimation(pair_counts)
        self.transition_estimation(trans_counts)
        self.init_estimation(init_counts)
    def viterbi(self, obs):
        """Viterbi algorithm:
        Find the states corresponding to the oberservations.
        The oberservations must be converted in a list of indices."""
        if len(obs)<2:
            # the length of observation is smaller strictly than 2
            return []
        # shortcuts about this class's functions
        B = self.observation_proba
        A = self.transition_proba
        T = len(obs)
        N = self.N
        # initialisation
        # init
        delta = zeros(N**2, float)
        tmp = zeros(N**2, float)
        psi = zeros((T, N**2), int)
        delta_t = zeros(N**2, float)
        # apply initial_state probs to the first frame
        delta = B[obs[1]] * self.initial_state_proba
        # recursion
        for t in range(2, T):
            O_t = obs[t]
            for j in range(N):
                for k in range(N):
                    # tmp[0:N] represent 0<=i<=N-1
                    tmp = delta[j::N] * A[k, j::N] # one by one multiply
                    id_i = psi[t, j*N+k] = tmp.argmax()
                    delta_t[j*N+k] = tmp[id_i] * B[O_t, j*N+k]
            delta, delta_t = delta_t, delta
        # reconstruction
        # find j, k s.t. delta[j*N+k] is the maximal value(delta.max()).
        j_quotient, k_reminder = divmod(delta.argmax(), N)
        # construct the list of tags by matrix psi
        i_star = [j_quotient, k_reminder]
        for psi_t in psi[-1:1:-1]:
            i_star[:0] = [psi_t[i_star[0] * N + i_star[1]]]
        return i_star

## Compute what Hmm needs
- The state transition probability distribution
- The obervation symbol probability distribution
- The initial state distribution 

In [31]:
def make_counts(corpus):
    """
    Build different count tables to train a HMM_2. Each count table is a dictionnary.
    Returns:
    * c_letters: letters counts
    * c_tags: tag counts
    * c_pairs: count of pairs (pre_tag, tag, letter)
    * c_transitions: count of tag 3-gram
    * c_inits: count of 2-gram found in the first and second position
    """
    c_letters = dict()
    c_tags = dict()
    c_pairs = dict()
    c_transitions = dict()
    c_inits = dict()
    for word in corpus:
        # we use i because of the transition counts
        for i in range(len(word)):
            couple = word[i]
            letter = couple[0]
            tag = couple[1]
            # word counts
            if letter in c_letters:
                c_letters[letter] = c_letters[letter] + 1
            else:
                c_letters[letter] = 1
            # tag counts
            if tag in c_tags:
                c_tags[tag] = c_tags[tag] + 1
            else:
                c_tags[tag] = 1
            if i >= 2:
                # transition counts, z is combination of two previous states
                z = (word[i - 2][1], word[i - 1][1])
                trans = (z, tag)
                if trans in c_transitions:
                    c_transitions[trans] = c_transitions[trans] + 1
                else:
                    c_transitions[trans] = 1
                # observation counts
                o = ((word[i-1][1], tag), letter)
                if o in c_pairs:
                    c_pairs[o] = c_pairs[o] + 1
                else:
                    c_pairs[o] = 1
            if i == 1:
                # init counts, i == 1 -> counts for initial states
                z = (word[i - 1][1], tag)
                if z in c_inits:
                    c_inits[z] = c_inits[z] + 1
                else:
                    c_inits[z] = 1
                # observation counts
                o = ((word[i - 1][1], tag), letter)
                if o in c_pairs:
                    c_pairs[o] = c_pairs[o] + 1
                else:
                    c_pairs[o] = 1
            # i == 0 -> To present the bound, we can insert special symbol if we want
            if i == 0:
                continue
    return c_letters, c_tags, c_pairs, c_transitions, c_inits

## Creation of vocabulary according to the number of occurence for each letter.

In [32]:
def make_vocab(c_letters, threshold):
    """
    return a vocabulary by thresholding letter counts.
    inputs:
    * c_letters : a dictionnary that maps letter to its counts
    * threshold: count must be >= to the threshold to be included
    returns:
    * a letter list
    """
    voc = list()
    voc.append(UNK)
    for l in c_letters:
        if c_letters[l] >= threshold:
            voc.append(l)
    return voc

## Loading two data sets


In [33]:
# read train data
f_in = open("./data/train10.pkl", "rb")
data_train10 = pickle.load(f_in)
f_in.close()
# print len(data_train10)
# read test data
f_in = open("./data/test10.pkl", "rb")
data_test10 = pickle.load(f_in)
f_in.close()
print(data_train10[10:13])
cletters, ctags, cpairs, ctrans, cinits = make_counts(data_train10)
# print "Nombre de letters  : " + str(len(cletters))
# print "Nombre de tags  : " + str(len(ctags))
# print "Nombre de paires: " + str(len(cpairs))
# print "Nombre de trans : " + str(len(ctrans)) + " / " + str(26 * 26 * 26)
# print "Nombre de init. : " + str(len(cinits))

vocab = make_vocab(cletters, 1)
vocab.sort()
# print "----------%d vocabulaire----------" % len(vocab)
# print vocab

states = list(ctags.keys())
states.sort()
# print "----------%d states----------" % len(states)
# print states

[[('o', 'o'), ('f', 'f')], [('p', 'l'), ('j', 'i'), ('b', 'b'), ('e', 'e'), ('r', 'r'), ('a', 'a'), ('t', 't'), ('i', 'i'), ('o', 'o'), ('n', 'n')], [('i', 'i'), ('n', 'n')]]


## Creation of HMM_2

In [34]:
hmm = HMM_2(state_list=states, observation_list=vocab,
            transition_proba=None,
            observation_proba=None,
            initial_state_proba=None)
hmm.supervised_training(cpairs, ctrans, cinits)

HMM_2 creating with: 
26 states
27 observations
Y_index:
 {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
X_index:
 {'<unk>': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


## Calculating three neccessary probability distributions one by one 

In [35]:
hmm.observation_estimation(cpairs)

In [36]:
hmm.transition_estimation(ctrans)

In [37]:
hmm.init_estimation(cinits)

## Training data one time(we can get three neccessary matrix once).

In [38]:
# hmm.supervised_training(cpairs, ctrans, cinits)
tot=0.0
correct=0.0
for word in data_test10:
    letter_index, tag_index = hmm.data2indices(word)
    pre_tag_index = hmm.viterbi(letter_index)
    correct += np.count_nonzero(np.array(tag_index) == np.array(pre_tag_index))
    tot+=len(word)
print("The accuracy(%) : "+str(correct)+" / "+str(tot)+ " -> "+ str(correct*100/tot))

The accuracy(%) : 6292.0 / 7320.0 -> 85.95628415300547
